Function for returning the probabilities of a PR-box.
Later we want to use the two possible outputs {-1,+1}. However the simplest definition of the PR box probability
distribution is using outputs {0,1}. Hence we just reassign the output -1 to 0 in within the function, as this does not
influence the probabilities of the PR box.
TODO: Insert $$\LaTeX$$ equations.

In [22]:
import numpy as np
def prob_pr_box(a,b,x,y):
    s = x*y
    if a == -1: a = 0
    if b == -1: b = 0
    t = (a+b) % 2
    return 1/2 * int(s == t)

Now we iterate through all possible inputs and outputs to define the behavior corresponding to the PR box

In [23]:
import itertools
inputs = [0,1]
outputs = [-1,1]
dim = len(outputs) ** (2*len(inputs))
p = []

for x,y in itertools.product(inputs,inputs):
    for a,b in itertools.product(outputs,outputs):
        p.append(prob_pr_box(a,b,x,y))

p = np.array(p)
assert len(p) == dim

Now we have the behavior defined as a vector. As we have shown before (see blackboard) the
deterministic behaviors are just the basis vectors of R^16.

In [24]:
deterministics = []
for i in range(dim):
    d = np.zeros(dim)
    d[i] = 1
    deterministics.append(d)

Now we can come to the linear optimization problem to find the bell inequality. Therefore we need to make
some adaptions to the dual formulation of the problem as we want to use SciPy to solve the problem.

We extend the dimensions of the problem to '''dim+1''', where the last entry of the behavior and all the deterministic
behaviors is equal to -1. Thus we only need to find a '''dim+1''' dimensional vector that fulfills the inequalities
and all the subtractions are included in the vector product.

In [25]:
# modify behaviors for scipy optimization
for i in range(len(deterministics)):
    deterministics[i] = np.append(deterministics[i],[-1.0])
p = np.append(p, [-1.0])

Now define the inequalities and the objective function to run the SciPy optimizer. Note here that SciPy only provides a
minimizer. So we have to invert the sign of the objective function.

In [26]:
# objective function
obj = -p

# left hand side of the inequalities
lhs_ineq = deterministics
lhs_ineq.append(p)

# right hand side of the inequalities
rhs_ineq = np.zeros(dim+1)
rhs_ineq[-1] = 1

In [27]:
from scipy.optimize import linprog

# run the optimizer
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq)
print(opt.message)

Optimization terminated successfully.


Now we can retrieve the result from the optimal values. The last entry in the optimal vector is the bound variale S_l.
Multiplying p with the optimal result gives us the maximal value of S = s*p - S_l.
As this is 1, we know that the behavior is non local. To get the total value of s*p and S_l we have to seperate the
optimal values further

In [28]:
S = np.dot(p,opt.x)
print('S = {}'.format(S))
print('s*p = {}'.format(np.dot(p[:-1],opt.x[:-1])))
print('S_l = {}'.format(opt.x[-1]))

S = 1.0000000000019682
s*p = 2.314959538106127
S_l = 1.314959538104159


From what I've read I hear that the PR box violates the CHSH inequality with a maximal violation of 4. However with
this program I only find a violation of 2.33, which is even lower than 2 * sqrt(2) (which would be the maximal value
achievable with quantum behavior).

Maybe I should check what kind of inequality results here, as I also remember that for the local polytope the only
Bell inequality (that is non trivial) is the CHSH inequality. But maybe I am confusing something here.

So let's continue with check what violation of the CHSH inequality would give.

In [29]:
# calculate the value of the chsh inequality by simultaneously calculating each correlator
chsh = 0
for a,b in itertools.product(outputs,outputs):
    chsh += a*b*prob_pr_box(a,b,0,0)
    chsh += a*b*prob_pr_box(a,b,0,1)
    chsh += a*b*prob_pr_box(a,b,1,0)
    chsh -= a*b*prob_pr_box(a,b,1,1)
print(chsh)

4.0
